In [1]:

import dask
import dask.array as da
import torch
import numpy as np
tensor = torch.randn(501,300,20000)
chunks_size = (50,300,20000)
dask_array = da.from_array(tensor.numpy(), chunks=chunks_size)  
print(dask_array.shape)
import torch.nn as nn
model = nn.Sequential(nn.Flatten(),nn.Linear(in_features=300*200*100, out_features=10))  
def inference(chunk, model):
 
    model.eval()


    expected_shape = (chunk.shape[0], 300, 20000)
    assert chunk.shape == expected_shape, f"Invalid chunk shape: {chunk.shape}"

    with torch.no_grad():
        chunk_tensor = torch.from_numpy(chunk)
        chunk_flat = chunk_tensor.view(chunk_tensor.size(0), -1)
        output=model(chunk_flat)
        
    output=output.detach().numpy()
    output = output[:,np.newaxis]
    return output

result = dask_array.map_blocks(inference,model=model,dtype=np.float32,chunks = chunks_size)
result_array = result.compute()
#


(501, 300, 20000)


In [2]:
result_array.shape

(501, 1, 10)

In [4]:
result_serial = model(tensor)
result_serial.shape

torch.Size([500, 10])

In [4]:
!pip install h5py


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 29.8 MB/s eta 0:00:00m eta 0:00:010:01


In [4]:
import h5py
import torch

# Suppose your tensor is `tensor` (batch, channel, height, width)
tensor = torch.randn(500,60, 224, 224)

# Create an HDF5 file
with h5py.File('tensor_data.h5', 'w') as hf:
    # Create a dataset within the HDF5 file
    hf.create_dataset('data', data=tensor.numpy())


In [1]:
import h5py

# Ouverture du fichier HDF5 en mode lecture
with h5py.File('tensor_data.h5', 'r') as f:
    # Accéder au jeu de données
    dataset = f["./"]
    dataset_2= f["./"]
    dataset_3= f["./"]
    dataset_4= f["./"]

In [2]:
dataset.shape

AttributeError: 'Group' object has no attribute 'shape'

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import torch
sys.path.append('../src')
sys.path.append('../util')
from torch import sparse
from abstractModule import AbstractLinear as AL
from abstract import abstractTensor as AT

In [2]:
tensor_to_abstract = torch.randn(3,224,224)
abstract_tensor = AT(tensor_to_abstract,alpha=0.03*torch.ones(224*224)).abstract_tensor()
abstract_tensor.shape


torch.Size([50178, 3, 224, 224])

In [4]:
sys.getsizeof(abstract_tensor)

80

In [78]:

abstract_tensor = abstract_tensor.to_sparse()

In [60]:
sys.getsizeof(abstract_tensor)

80

In [61]:
abstract_tensor


tensor(indices=tensor([[    0,     0,     0,  ..., 50174, 50175, 50176],
                       [    0,     0,     0,  ...,     0,     0,     0],
                       [    0,     0,     0,  ...,   223,   223,   223],
                       [    0,     1,     2,  ...,   221,   222,   223]]),
       values=tensor([-0.3013,  1.0774,  1.2484,  ...,  0.0300,  0.0300,
                       0.0300]),
       size=(50178, 3, 224, 224), nnz=200704, layout=torch.sparse_coo)

In [99]:
indices = torch.arange(0,20)
indices = torch.tensor(indices)
result = abstract_tensor.index_select(0, indices)

print(result.shape)


torch.Size([20, 3, 224, 224])


/var/folders/52/k6pzm88n5_xdgm1mjpjcgnc80000gn/T/ipykernel_34780/2669809179.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indices = torch.tensor(indices)


In [107]:
device = torch.device('cpu')
abstract_tensor.to(device)

tensor(indices=tensor([[    0,     0,     0,  ..., 50174, 50175, 50176],
                       [    0,     0,     0,  ...,     0,     0,     0],
                       [    0,     0,     0,  ...,   223,   223,   223],
                       [    0,     1,     2,  ...,   221,   222,   223]]),
       values=tensor([-0.3013,  1.0774,  1.2484,  ...,  0.0300,  0.0300,
                       0.0300]),
       size=(50178, 3, 224, 224), nnz=200704, layout=torch.sparse_coo)

In [97]:
indices


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [105]:
result = result.to_dense()
print(result[0].shape)

result = torch.randn(20,3,224,224)

result = result.to_sparse()
abstract_tensor.index_put_([indices],result)

torch.Size([3, 224, 224])


NotImplementedError: Could not run 'aten::_index_put_impl_' with arguments from the 'SparseCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_index_put_impl_' is only available for these backends: [CPU, MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSn

In [13]:
abstract_tensor.indices()[1,:,:,:]

IndexError: too many indices for tensor of dimension 2